Use of OpenAI's Assistants API for the creation of a more advanced chat completion assistant.

For more information, check [OpenAI's documentation](https://platform.openai.com/docs/assistants/overview?context=with-streaming).

In [ ]:
!pip install --upgrade openai
!pip show openai | grep Version

Version: 1.14.0


In [ ]:
import json, time
from openai import OpenAI
from google.colab import userdata

In [ ]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [ ]:
client = OpenAI(
    base_url="https://open-assistant-ai.astra.datastax.com/v1",
    #ideally a paid API key
    api_key=userdata.get('OPENAI_API_KEY'),
    default_headers={
        "astra-api-token": userdata.get('assistant_api_db')
        })

assistant = client.beta.assistants.create(
    name="Finance Assistant",
    #Apply prompt engineering to receive better results, <xml> tags and such
    instructions="You are a sophisticated finance assistant. Your goal is to assist startup entrepeneurs with their finances, similar to an accountant's task. You will conduct intreviews in which you will ask the entrepeneurs for data about their finances, the goal of this interview is to fill in financial forms which you will retrieve from the entrepeneurs. Once you have asked the user to send a file of the form and you have interpreted it and filled it in, your last task will be to output the same form, only this time will be filled with the necessary information. You may also assist the entrepeneurs explaining the different terms and values that appear on said financial forms, and you may occasionally suggest to explain these terms to the entrepeneurs.",
    #ideally gpt-4-1106-preview or similar
    model="gpt-3.5-turbo",
)
show_json(assistant)

{'id': '37429366-e2be-11ee-ba31-96cf47d504ae',
 'created_at': 1710501709,
 'description': '',
 'file_ids': [],
 'instructions': "You are a sophisticated finance assistant. Your goal is to assist startup entrepeneurs with their finances, similar to an accountant's task. You will conduct intreviews in which you will ask the entrepeneurs for data about their finances, the goal of this interview is to fill in financial forms which you will retrieve from the entrepeneurs. Once you have asked the user to send a file of the form and you have interpreted it and filled it in, your last task will be to output the same form, only this time will be filled with the necessary information. You may also assist the entrepeneurs explaining the different terms and values that appear on said financial forms, and you may occasionally suggest to explain these terms to the entrepeneurs.",
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'Finance Assistant',
 'object': 'assistant',
 'tools': []}

Now we're going to make use of the most important feature of the Assistants API, "Threads", thanks to the threads our assistant will be able to maintain a state during the conversations, which won't require us to maintain the document retrieval, define the tools, or manage the state of the conversation each time the users send a new message.

In [ ]:
thread = client.beta.threads.create()
show_json(thread)

{'id': '3761337a-e2be-11ee-988b-32dc6582fe5e',
 'created_at': 1710501709,
 'metadata': {},
 'object': 'thread'}

In [ ]:
#link a new message to the thread created above for demo purposes
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to fill in a capital information form. Could you help me?",
)
show_json(message)

{'id': '37900e52-e2be-11ee-b4e3-d6d69b2eb7ad',
 'assistant_id': 'TODO',
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to fill in a capital information form. Could you help me?'},
   'type': 'text',
   'image_file': None}],
 'created_at': 1710501709,
 'file_ids': [],
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': 'None',
 'status': None,
 'thread_id': '3761337a-e2be-11ee-988b-32dc6582fe5e'}

Unlike the previous Chat Completions API, the Assistants API responds to the users with *Runs*, a run is created by assigning a Thread to an Assistant

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id, #'7a33ced2-e2b5-11ee-8854-32dc6582fe5e'
)
show_json(run)

{'id': '37beef92-e2be-11ee-a4d6-d6d69b2eb7ad',
 'assistant_id': '37429366-e2be-11ee-ba31-96cf47d504ae',
 'cancelled_at': 0,
 'completed_at': 0,
 'created_at': 1710501710,
 'expires_at': 0,
 'failed_at': 0,
 'file_ids': [],
 'instructions': "You are a sophisticated finance assistant. Your goal is to assist startup entrepeneurs with their finances, similar to an accountant's task. You will conduct intreviews in which you will ask the entrepeneurs for data about their finances, the goal of this interview is to fill in financial forms which you will retrieve from the entrepeneurs. Once you have asked the user to send a file of the form and you have interpreted it and filled it in, your last task will be to output the same form, only this time will be filled with the necessary information. You may also assist the entrepeneurs explaining the different terms and values that appear on said financial forms, and you may occasionally suggest to explain these terms to the entrepeneurs.",
 'last_er

In [ ]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress" or run.status=="generating":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [ ]:
run = wait_on_run(run, thread)
show_json(run)

{'id': '37beef92-e2be-11ee-a4d6-d6d69b2eb7ad',
 'assistant_id': '37429366-e2be-11ee-ba31-96cf47d504ae',
 'cancelled_at': 0,
 'completed_at': 0,
 'created_at': 1710501710,
 'expires_at': 0,
 'failed_at': 0,
 'file_ids': [],
 'instructions': "You are a sophisticated finance assistant. Your goal is to assist startup entrepeneurs with their finances, similar to an accountant's task. You will conduct intreviews in which you will ask the entrepeneurs for data about their finances, the goal of this interview is to fill in financial forms which you will retrieve from the entrepeneurs. Once you have asked the user to send a file of the form and you have interpreted it and filled it in, your last task will be to output the same form, only this time will be filled with the necessary information. You may also assist the entrepeneurs explaining the different terms and values that appear on said financial forms, and you may occasionally suggest to explain these terms to the entrepeneurs.",
 'last_er

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': '37dfa318-e2be-11ee-a4d6-d6d69b2eb7ad',
   'assistant_id': '37429366-e2be-11ee-ba31-96cf47d504ae',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "Of course, I'd be happy to help you fill out the capital information form. To get started, could you please send me the form so I can review it and gather the necessary information from you?"},
     'type': 'text',
     'image_file': None}],
   'created_at': 1710501710,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': '37beef92-e2be-11ee-a4d6-d6d69b2eb7ad',
   'status': None,
   'thread_id': '3761337a-e2be-11ee-988b-32dc6582fe5e'},
  {'id': '37900e52-e2be-11ee-b4e3-d6d69b2eb7ad',
   'assistant_id': 'TODO',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to fill in a capital information form. Could you help me?'},
     'type': 'text',

The Assistant's response is the following:

In [ ]:
print(messages.data[0].content[0].text.value)

Of course, I'd be happy to help you fill out the capital information form. To get started, could you please send me the form so I can review it and gather the necessary information from you?


Let's continue the conversation to ensure that the Thread is working properly and maintaining the session's context

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="I don't have the form at the moment, I will send you later. Meanwhile could you explain to me what a cash reserve is and how should I measure a proper amount for my company?"
)


run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


wait_on_run(run, thread)

# Ideally we woulwd wetrieve only the messages added after our last user message, but we don't have pagination enabled for this API endpoint yet
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc",
    #after=message.id
)
show_json(messages)

{'data': [{'id': '37900e52-e2be-11ee-b4e3-d6d69b2eb7ad',
   'assistant_id': 'TODO',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to fill in a capital information form. Could you help me?'},
     'type': 'text',
     'image_file': None}],
   'created_at': 1710501709,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': 'None',
   'status': None,
   'thread_id': '3761337a-e2be-11ee-988b-32dc6582fe5e'},
  {'id': '37dfa318-e2be-11ee-a4d6-d6d69b2eb7ad',
   'assistant_id': '37429366-e2be-11ee-ba31-96cf47d504ae',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "Of course, I'd be happy to help you fill out the capital information form. To get started, could you please send me the form so I can review it and gather the necessary information from you?"},
     'type': 'text',
     'image_file': None}],
   'creat

In [ ]:
print(messages.data[-1])

Message(id='39c25bc6-e2be-11ee-a4d6-d6d69b2eb7ad', assistant_id='37429366-e2be-11ee-ba31-96cf47d504ae', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Certainly! A cash reserve is the amount of money that a company sets aside to cover unexpected expenses or to weather financial difficulties. It acts as a safety net to ensure that the business can continue operating smoothly even during challenging times.\n\nTo determine the appropriate amount for your company's cash reserve, you should consider factors such as your monthly expenses, revenue streams, and any potential risks or uncertainties in your industry. A common rule of thumb is to have enough cash reserves to cover at least three to six months of operating expenses.\n\nIt's important to regularly assess and adjust your cash reserve based on changes in your business environment to ensure that you always have an adequate financial buffer."), type='text', image_file=None)], created_at=1710501713, file_i

In [ ]:
print(messages.data[-1].content[0].text.value)

Certainly! A cash reserve is the amount of money that a company sets aside to cover unexpected expenses or to weather financial difficulties. It acts as a safety net to ensure that the business can continue operating smoothly even during challenging times.

To determine the appropriate amount for your company's cash reserve, you should consider factors such as your monthly expenses, revenue streams, and any potential risks or uncertainties in your industry. A common rule of thumb is to have enough cash reserves to cover at least three to six months of operating expenses.

It's important to regularly assess and adjust your cash reserve based on changes in your business environment to ensure that you always have an adequate financial buffer.


In [ ]:
assistant.id

'37429366-e2be-11ee-ba31-96cf47d504ae'

In [ ]:
FINANCE_ASSISTANT_ID = assistant.id

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [ ]:
def pretty_print(messages):
    print("-- Messages")
    for m in messages.data:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [ ]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(FINANCE_ASSISTANT_ID, thread, user_input)
    return thread, run


# I'm going to simulate concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need help understanding the financial laws of Togo, and the financial requirements for creating a startup."
)
thread2, run2 = create_thread_and_run("Could you help me fill out this form?")
thread3, run3 = create_thread_and_run("I don't understand most of the financial terms for a document I need to submit. What can I do?")

In [ ]:

run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))


run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))


run4 = submit_message(FINANCE_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

-- Messages
user: I need help understanding the financial laws of Togo, and the financial requirements for creating a startup.
assistant: I can definitely help you with that. To assist you better, I would need some specific information about your startup. For example, the nature of your business, expected revenue, planned expenses, and any other relevant financial details. That way, I can guide you through the financial requirements for creating a startup in Togo and help you ensure compliance with the financial laws.

Do you have any financial documents or forms that you have already started filling out for your startup in Togo? It would be helpful to review those so I can provide more tailored assistance.

-- Messages
user: Could you help me fill out this form?
assistant: Of course! I'd be happy to help. Please go ahead and send me the form so I can take a look at it.

-- Messages
user: I don't understand most of the financial terms for a document I need to submit. What can I do?
ass

Now that the concurrent Threads have successfully been tested, I'm going to update our Assistant with the tools needed to process code and documents

In [ ]:
assistant = client.beta.assistants.update(
    FINANCE_ASSISTANT_ID,
    tools=[{"type": "retrieval"}],
)
print(assistant.tools)

[RetrievalTool(type='retrieval', function=None)]


In [ ]:
form = client.files.create(
    file=open(
        "/content/capital_requirements_2.csv",
        "rb",
    ),
    purpose="assistants",
)

In [ ]:
assistant = client.beta.assistants.update(
    FINANCE_ASSISTANT_ID,
    file_ids=[form.id],
)
print(assistant.file_ids)

['bbb90fee-e2cd-11ee-8854-32dc6582fe5e']


In [ ]:
thread, run = create_thread_and_run(
    "Can you help me understand the fields of this CSV form?."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

-- Messages
user: Can you help me understand the fields of this CSV form?.
assistant: Of course! The form you provided contains fields related to the capital requirements and sources of financing for a business. Here is a breakdown of the fields mentioned in the form:

1. **Capital Requirements**:
   - **Investments**:
     - Immaterial goods
     - Registration costs
     - Machinery and equipment (IT)
     - Production equipment acquirements
     - Assets, in kind
     - Vehicle
     - Furniture
     - Phone/fax/Internet including installation
     - Installation of machinery and equipment
     - Renovation of premises
     - Office supplies
     - Other devices
     - Initial advertising/brochures
     - Rent and rent deposit

2. **Sources of Financing**:
   - **Capital Sources**:
     - Share capital
     - Personal funds invested
     - Personal production tools and equipment
     - Equity loan (Shareholder loan, Bank loan, Finnvera loan, Supplier loan)
   - **Other Financial Asse

In [ ]:
new_message="Thank you, that's very helpful, help me fill out this form, going one by one with these fields, explaining each term and letting me know if the value I submit strays too far from what a reasonable value could be?"
new_run = submit_message(FINANCE_ASSISTANT_ID, thread, new_message)
new_run = wait_on_run(new_run, thread)
pretty_print(get_response(thread))

-- Messages
user: Can you help me understand the fields of this CSV form?.
assistant: Of course! The form you provided contains fields related to the capital requirements and sources of financing for a business. Here is a breakdown of the fields mentioned in the form:

1. **Capital Requirements**:
   - **Investments**:
     - Immaterial goods
     - Registration costs
     - Machinery and equipment (IT)
     - Production equipment acquirements
     - Assets, in kind
     - Vehicle
     - Furniture
     - Phone/fax/Internet including installation
     - Installation of machinery and equipment
     - Renovation of premises
     - Office supplies
     - Other devices
     - Initial advertising/brochures
     - Rent and rent deposit

2. **Sources of Financing**:
   - **Capital Sources**:
     - Share capital
     - Personal funds invested
     - Personal production tools and equipment
     - Equity loan (Shareholder loan, Bank loan, Finnvera loan, Supplier loan)
   - **Other Financial Asse